In [ ]:
import warnings
warnings.filterwarnings('ignore')
import pandas as pd, numpy as np
import matplotlib.pyplot as plt

In [ ]:
df=pd.read_csv('../input/logistic-regression/Social_Network_Ads.csv')

In [ ]:
df.head()

In [ ]:
df.info()

In [ ]:
gen= pd.get_dummies(df.Gender, drop_first=True)
df=pd.concat([df,gen], axis=1)
df.drop('Gender',axis=1, inplace=True)
df.head()

# Model Building

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X= df.drop(['Purchased','User ID'], axis=1)
X.head()

In [ ]:
y= df.Purchased
y.head()

In [ ]:
# train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.7, test_size=0.3, random_state=100)

In [ ]:
from sklearn.preprocessing import StandardScaler

In [ ]:
scaler= StandardScaler()

In [ ]:
X_train[['Age','EstimatedSalary']]=scaler.fit_transform(X_train[['Age','EstimatedSalary']])

In [ ]:
X_train.head()

In [ ]:
purchase_rate= (sum(df['Purchased'])/len(df['Purchased']))*100
purchase_rate

In [ ]:
import statsmodels.api as sm

In [ ]:
lm01= sm.GLM(y_train,(sm.add_constant(X_train)), family= sm.families.Binomial())
lm01=lm01.fit()
lm01.summary()

In [ ]:
X_train_sm= sm.add_constant(X_train)

In [ ]:
y_train_pred=lm01.predict(X_train_sm)

In [ ]:
y_train_pred=y_train_pred.values.reshape(-1)
y_train_pred[:10]

In [ ]:
y_train_pred_final= pd.DataFrame({'Purchase':y_train.values, 'Purchase_prob':y_train_pred})
y_train_pred_final['User ID']=y_train.index
y_train_pred_final.head()

In [ ]:
numbers = [float(x)/10 for x in range(10)]
for i in numbers:
    y_train_pred_final[i]= y_train_pred_final.Purchase_prob.map(lambda x: 1 if x > i else 0)
y_train_pred_final.head()

In [ ]:
# Now let's calculate accuracy sensitivity and specificity for various probability cutoffs.
cutoff_df = pd.DataFrame( columns = ['prob','accuracy','sensi','speci'])
from sklearn import metrics
from sklearn.metrics import confusion_matrix

# TP = confusion[1,1] # true positive 
# TN = confusion[0,0] # true negatives
# FP = confusion[0,1] # false positives
# FN = confusion[1,0] # false negatives

num = [0.0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9]
for i in num:
    cm1 = metrics.confusion_matrix(y_train_pred_final.Purchase, y_train_pred_final[i] )
    total1=sum(sum(cm1))
    accuracy = (cm1[0,0]+cm1[1,1])/total1
    
    speci = cm1[0,0]/(cm1[0,0]+cm1[0,1])
    sensi = cm1[1,1]/(cm1[1,0]+cm1[1,1])
    cutoff_df.loc[i] =[ i ,accuracy,sensi,speci]
print(cutoff_df)

In [ ]:
cutoff_df.plot.line(x='prob', y=['accuracy','sensi','speci'])
plt.show()

In [ ]:
# from the above graph we are choosing 0.4 as the cut-off probability
y_train_pred_final['final_predicted']=y_train_pred_final.Purchase_prob.map( lambda x: 1 if x > 0.4 else 0)
y_train_pred_final.head()

In [ ]:
y_train_pred_final.drop([0.0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9], axis=1, inplace=True)
y_train_pred_final.head()

In [ ]:
#accuracy
metrics.accuracy_score(y_train_pred_final.Purchase, y_train_pred_final.final_predicted)

In [ ]:
confusion01= metrics.confusion_matrix(y_train_pred_final.Purchase, y_train_pred_final.final_predicted)
confusion01

In [ ]:
TP = confusion01[1,1] # true positive 
TN = confusion01[0,0] # true negatives
FP = confusion01[0,1] # false positives
FN = confusion01[1,0] # false negatives

In [ ]:
# Let's see the sensitivity of our logistic regression model
TP / float(TP+FN)

In [ ]:
# Let us calculate specificity
TN / float(TN+FP)